In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt

In [2]:
grid = gpd.read_file('grid_1mile.json').assign(
    idx = lambda x: 'hex-' + x.index.astype(str)
)

In [3]:
dat_points = pd.read_csv('priority1calls.csv').assign(
    geometry = lambda x: x.apply(
        lambda row: Point(
            row['Incident_Longitude'],
            row['Incident_Latitude']
        ), axis=1
    )
)

In [4]:
dat_points = gpd.GeoDataFrame(
    dat_points
)

dat_points.crs = grid.crs

In [5]:
dat_points_grid = gpd.sjoin(
    dat_points,
    grid,
    how='left',
    op='within'
)

In [6]:
breakdown_priority = dat_points_grid[[
    'idx',
    'Incident_Priority'
]].pivot_table(
    index='idx',
    columns='Incident_Priority',
    aggfunc=pd.Series.count
).apply(
    lambda x: round(
        (x / sum(x)) * 100,
        2
    ), axis=1
).fillna(0).rename(
    columns = lambda x: 'priority-' + str(x)
)

In [7]:
breakdown_seconds = dat_points_grid.groupby('idx')['Incident_Priority'].agg([
        pd.np.size
    ]
)

In [8]:
breakdown_goal = dat_points_grid.groupby('idx')[
    'Response_Interval_Goal_Met (1=Y; " "=N)'
].agg([
    sum,
    pd.Series.count
]).rename(
    columns={
        'sum': 'goal_met',
        'count': 'total_count'
    }
).assign(
    pct_goal_met = lambda x: round(
        (x['goal_met'] / x['total_count']) * 100,
        2
    )
)

In [9]:
grid_merge = grid.set_index('idx').join(
    breakdown_seconds.join(
        breakdown_priority
    ).join(
        breakdown_goal
    )
).fillna(0).query('size > 0').rename(
    columns = lambda x: str(x)
)

In [10]:
grid_merge.to_file(
    'points-grid_1mile.geojson',
    driver='GeoJSON'
)